
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv('data/Corona_NLP_train.csv', encoding="ISO-8859-1")

# Filter out tweets with Sentiment = Neutral
data = data[data['Sentiment'] != 'Neutral']

# Split into X and Y
X = np.array(data['OriginalTweet'])
Y = np.where((data['Sentiment'] == 'Positive') | (data['Sentiment'] == 'Extremely Positive'), 1, 0)

# Split into train (0.6), test (0.15), and validation (0.25) sets without randomizing
n_samples = len(X)
train_end = int(n_samples * 0.6)
test_end = train_end + int(n_samples * 0.15)

X_train, X_test, X_valid = X[:train_end], X[train_end:test_end], X[test_end:]
Y_train, Y_test, Y_valid = Y[:train_end], Y[train_end:test_end], Y[test_end:]

In [2]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


model = Pipeline([('vectorizer', CountVectorizer()), ('model', LogisticRegression(max_iter=5000))])
model.fit(X_train, Y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('model', LogisticRegression(max_iter=5000))])

In [3]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

from sklearn.metrics import precision_score, recall_score

Y_pred = model.predict(X_test)

precision_0 = precision_score(Y_test, Y_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)

In [4]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost

    # Get predicted probabilities
    y_proba = model.predict_proba(X)

    # Get predictions based on threshold
    y_pred = np.where(y_proba[:, 1] > float(threshold), 1, 0)

    #cost = 0

    # Iterate through predictions and true labels
    #for i in range(0, y_pred):
        # Check for misclassification
    #    if y_pred[i] != Y[i]:
            # Positive tweet classified as negative
    #        if Y[i] == 1:
    #            cost += 1
            # Negative tweet classified as positive
    #        elif Y[i] == 0:
    #            cost += 5

    # Cost of positive tweets classified as negative (FN)
    cost_fn = np.sum((Y == 1) & (y_pred != 1)) * 1

    # Cost of negative tweets classified as positive (FP)
    cost_fp = np.sum((Y == 0) & (y_pred != 0)) * 5

    # Average cost
    avg_cost = (cost_fn + cost_fp) / len(Y)
    
    return avg_cost

In [5]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

costs = {}
for threshold in np.arange(0.1, 1, 0.1):
    c = cost(model, threshold, X_test, Y_test)
    costs[threshold] = c

optimal_threshold = float(min(costs, key=costs.get))
cost_at_optimal_threshold = float(min(costs.values()))

In [6]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

# Bounds for the random variable (cost per sample)
a = 0
b = 5

# Delta for the given confidence interval
delta = 1 - 0.99

# Hoeffding's epsilon
epsilon = np.sqrt(((b - a)**2 * np.log(2/delta)) / (2 * len(Y_valid)))

# Confidence interval
l_bound = cost_at_optimal_threshold_valid - epsilon
u_bound = cost_at_optimal_threshold_valid + epsilon

cost_interval_valid = (l_bound, u_bound)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [7]:

# Part 7

# Values of C for the validation set
y_proba_valid = model.predict_proba(X_valid)
y_pred_valid = np.where(y_proba_valid[:, 1] >= optimal_threshold, 1, 0)

C_values = (1 - (y_proba_valid[:, 1] >= optimal_threshold)) * Y_valid + 5 * (1 - Y_valid) * (y_proba_valid[:, 1] >= optimal_threshold)

# Empirical mean and variance of C
mean_C = np.mean(C_values)
variance_of_C = np.var(C_values)

# Bennett's inequality parameters
n_valid = len(Y_valid)
a = 0
b = 5
delta = 1 - 0.99

# Compute epsilon using Bennett's inequality
epsilon = np.sqrt((2 * variance_of_C * np.log(2/delta)) / n_valid) + ((b - a) * np.log(2/delta)) / (3 * n_valid)

# Confidence interval
l_bound = mean_C - epsilon
u_bound = mean_C + epsilon

interval_of_C = (l_bound, u_bound)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)